In [ ]:
import os
import sys
sys.path.append('../..')
import numpy as np
import deepbayesHF
import deepbayesHF.optimizers as optimizers
from deepbayesHF import PosteriorModel
from deepbayesHF.analyzers import FGSM
from deepbayesHF.analyzers import eps_LRP
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import cv2
import random
import matplotlib.pyplot as plt
from collections import namedtuple

import subprocess
from statistics import mode
import json

In [ ]:
def display_number(X,SCALE):
    X = X.reshape(SCALE[0],SCALE[1],3)
    plt.imshow(X,cmap='gray')
    plt.show()
    
def display_explanation(x0,SCALE,expl):
    fig, (ax1,ax2,ax3) = plt.subplots(1,3)
    x0_2d = x0.reshape(SCALE[0],SCALE[1],3)
    ax1.set_title('Input')
    ax1.imshow(x0_2d,cmap='gray')
    expl_2d = expl.reshape(SCALE[0],SCALE[1],3)
    ax2.set_title('Explanation')
    ax2.imshow(expl_2d)
    
    cmap = plt.cm.gray
    norm = plt.Normalize(x0.min(),x0.max())
    rgba = cmap(norm(x0))
    
    norm_ex = plt.Normalize(expl.min(),expl.max())
    normed_ex = norm_ex(expl)
    
    for i,x in enumerate(normed_ex):
        if x > 0:
            rgba[i] = x,0,0,1
    
    rgba = rgba.reshape(SCALE[0],SCALE[1],4)
    
    ax3.set_title('Overlayed')
    ax3.imshow(rgba,interpolation='nearest')
    plt.show()

In [ ]:
# Load mnist data and scale down to SCALE (trying 14x14 initially)

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
SCALE = (14,14)
if not SCALE == X_train[0].shape:
    X_train = np.array(list(map(lambda x:cv2.resize(x,SCALE,interpolation=cv2.INTER_CUBIC),X_train)))
X_train = np.array(list(map(lambda x:cv2.cvtColor(x,cv2.COLOR_GRAY2RGB),X_train)))
X_train = X_train/255.
X_train = X_train.astype("float32").reshape(len(X_train), SCALE[0],SCALE[1],3)

if not SCALE == X_test.shape:
    X_test = np.array(list(map(lambda x:cv2.resize(x,SCALE,interpolation=cv2.INTER_CUBIC),X_test)))
X_test = np.array(list(map(lambda x:cv2.cvtColor(x,cv2.COLOR_GRAY2RGB),X_test)))
X_test = X_test/255.
X_test = X_test.astype("float32").reshape(len(X_test), SCALE[0], SCALE[1],3)

# # make it a binary classification task (X or not X)
# target = 8
# y_train = np.array([1 if y == target else 0 for y in y_train])
# y_test = np.array([1 if y == target else 0 for y in y_test])

# # filter so we get about 50% target class and 50% other
# target_idxs = [i for i in range(len(y_train)) if y_train[i] == 1]
# other_idxs = [i for i in range(len(y_train)) if y_train[i] == 0]

# #uncomment the shuffle for training only
# #random.shuffle(other_idxs)
# other_idxs = other_idxs[:len(target_idxs)]
# # pick len(target_idxs) samples from the other_idxs
# X_train = np.array([x for i,x in enumerate(X_train) if i in other_idxs or i in target_idxs])
# y_train = np.array([y for i,y in enumerate(y_train) if i in other_idxs or i in target_idxs])

In [ ]:
model_name = f'mnist{SCALE[0]}x{SCALE[1]}_32_16_coverage_rgb_tmp'
opt = optimizers.VariationalOnlineGuassNewton()
likelihood = tf.keras.losses.SparseCategoricalCrossentropy()

inputs = Input(shape=X_train[0].shape)
tmp = Flatten()(inputs)
tmp = Dense(256,activation='relu')(tmp)
tmp = Dense(128,activation='relu')(tmp)
predictions = Dense(10,activation='softmax')(tmp)
model = Model(inputs=inputs,outputs=predictions)

bayes_model = opt.compile(model,loss_fn=likelihood,
                          epochs=5, learning_rate=0.25,
                          inflate_prior=2.0, log_file='tmp/log.txt')
bayes_model.train(X_train,y_train,X_test,y_test)
bayes_model.save(model_name)

In [ ]:
model_name = f'mnist{SCALE[0]}x{SCALE[1]}_32_16_coverage_rgb_tmp'
bayes_model = PosteriorModel(model_name)
y_pred = bayes_model.predict(X_test,n=50)
check_accuracy = tf.keras.metrics.Accuracy(name="train_acc")
check_accuracy(y_test,np.argmax(y_pred,axis=1))
print()
print('Loaded model accuracy:',f'{check_accuracy.result().numpy()*100:.2f}%')

In [ ]:
N = 50
# pick a random positive input for testing
n = 0

while True:
    n = np.random.choice(list(range(len(X_train))))
    if y_train[n] == 8:
        break


X = X_train[n].reshape(1,SCALE[0],SCALE[1],3).astype(np.float32)
y_hat = np.argmax(bayes_model.predict(X,n=N))
display_number(X,SCALE)
print('Prediction:',y_hat)
input_shape = X.flatten().shape
print(n)

In [ ]:
# naive method of generating a bayesian explanation

sys.path.append("../../LayerwiseRelevancePropagation/src")
from lrp import RelevancePropagation
import math


exps = []
X = X_train[n]
X = X.reshape(1,SCALE[0],SCALE[1],3)
for j in range(50):
    bayes_model.set_weights(bayes_model.sample())
    y = np.argmax(np.array(bayes_model._predict(X)).flatten())

    lrp = RelevancePropagation(0.01,'max',True,X.shape,bayes_model.model)
    exp = lrp.run(X.reshape(*X.shape[1:]))
    #print(exp)
    #plt.imshow(exp,vmin=0,vmax=1)
    #plt.show()

    exp_pop_flat = np.array([i for i in exp.flatten().flatten() if i > 0])
    max_rel = np.max(exp_pop_flat)
    av_rel = np.mean(exp_pop_flat)
    var_rel = np.var(exp_pop_flat)
    #print('Over populated spaces:')
    #print('Max:',max_rel,'Average:',av_rel,'Variance:',var_rel)

    limit = 0.6*max_rel
    #limit = 0
    exp[exp < limit] = 0
    exp[exp > 0] = 1
    #plt.imshow(exp)
    #plt.show()
    exps.append(exp)


In [ ]:
# visualise results
cmap = dict()
names = []
es = []
for e in exps:
    if not str(e) in names:
        names.append(str(e))
        es.append(e)
        cmap[names.index(str(e))] = 0
    cmap[names.index(str(e))] += 1

res = max(cmap,key=cmap.get)
res_image = es[res]
plt.imshow(res_image)
plt.axis('off')
plt.show()
cov = (cmap[res]/50)*100

print("P_cover:",cov)

In [ ]:
# better way of generating a bayesian explanation

sys.path.append("../../LayerwiseRelevancePropagation/src")
from lrp import RelevancePropagation
import math

from memo import memo

@memo
def generate_min_exps(expl,threshold):
    exps = []
    for i in range(len(expl)):
        orig_expl = expl
        if orig_expl[i] == 0:
            continue
        else:
            if i == len(expl) - 1:
                s = sum(orig_expl[:i])
            else:
                s = sum(orig_expl[:i])+sum(orig_expl[i+1:])
            if s < threshold:
                exps.append(expl)
                break
            else:
                new_expl = tuple(orig_expl[:i]) + (0,)
                if i < len(expl)-1:
                    new_expl = new_expl + tuple(orig_expl[i+1:])
                new_exps = generate_min_exps(new_expl,threshold)
                exps += new_exps
    return exps
exps = []
X = X_train[n]
X = X.reshape(1,SCALE[0],SCALE[1],3)
for j in range(2):
    bayes_model.set_weights(bayes_model.sample())
    X = X.reshape(1,SCALE[0],SCALE[1],3)
    y = np.argmax(np.array(bayes_model._predict(X)).flatten())

    lrp = RelevancePropagation(0.05,'max',True,X.shape,bayes_model.model)
    exp = lrp.run(X.reshape(*X.shape[1:]))
    #plt.imshow(exp,vmin=0,vmax=1)
    #plt.show()

    exp_pop_flat = np.array([i for i in exp.flatten().flatten() if i > 0])
    max_rel = np.max(exp_pop_flat)
    av_rel = np.mean(exp_pop_flat)
    var_rel = np.var(exp_pop_flat)

    limit =0.3*max_rel
    exp[exp < limit] = 0

    exp_list = set(generate_min_exps(tuple(exp.flatten()),0.9*np.sum(exp)))
    exps += exp_list


In [ ]:
# visualise results
print(len(exps))
cmap = dict()
names = []
es = []
for e in exps:
    e = np.array(e)
    e[e > 0 ] = 1
    if not str(e) in names:
        names.append(str(e))
        es.append(e)
        cmap[names.index(str(e))] = 0
    cmap[names.index(str(e))] += 1

res = max(cmap,key=cmap.get)
res_image = es[res]
plt.imshow(np.array(res_image).reshape(SCALE[0],SCALE[1]))
plt.axis('off')
plt.show()
cov = (cmap[res]/50)*100

print("P_cover:",cov)